In [3]:
# MODULE IMPORTS 
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [4]:
# custom function to extract joint angle

def calcAngle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    angle = angle % 360.0
    
    return angle

In [ ]:
# VIDEO FEED CAPTURE

cap = cv2.VideoCapture(0)                                  #captures video feed from the default camera

# INSTANCE SETUP                                             initialises the pose model
with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    
    count = 0
    knee_down = False
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # frame scaling factor
        frame = cv2.resize(frame, (0, 0), fx=1.5, fy=1.5)  

        
        # color format toggle for cv2
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        
        # process detection
        results = pose.process(image)

        
        # color format toggle for display
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        detection = True
        
        #extract landmarks
        if results.pose_landmarks is not None:
            landmarks = results.pose_landmarks.landmark
            detection = True
        else:
            cv2.putText(image, 
                         f"PERSON NOT DETECTED",
                        (820, 550),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (0,0,255),
                        2,
                        cv2.LINE_AA
                           )
            detection = False
            #continue
        
        
        # side select                                                        #performs left right shoulder comparison
        left_shoulder_x = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z
        right_shoulder_x = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z
        
        if left_shoulder_x < right_shoulder_x:
            side = "LEFT"
            jointsArray = ["LEFT_SHOULDER", "LEFT_HIP", "LEFT_KNEE", "LEFT_ANKLE", "LEFT_FOOT_INDEX"]
        elif left_shoulder_x > right_shoulder_x: 
            side = "RIGHT"
            jointsArray = ["RIGHT_SHOULDER", "RIGHT_HIP", "RIGHT_KNEE", "RIGHT_ANKLE", "RIGHT_FOOT_INDEX"]
        else:
            side = None

        
        # landmark co-ordinates and joint angle calculation
        try:
            lm_Shoulder = [landmarks[getattr(mp_pose.PoseLandmark, jointsArray[0]).value].x,
                           landmarks[getattr(mp_pose.PoseLandmark, jointsArray[0]).value].y]
            lm_Hip      = [landmarks[getattr(mp_pose.PoseLandmark, jointsArray[1]).value].x,
                           landmarks[getattr(mp_pose.PoseLandmark, jointsArray[1]).value].y]
            lm_Knee     = [landmarks[getattr(mp_pose.PoseLandmark, jointsArray[2]).value].x,
                           landmarks[getattr(mp_pose.PoseLandmark, jointsArray[2]).value].y]
            lm_Ankle    = [landmarks[getattr(mp_pose.PoseLandmark, jointsArray[3]).value].x,
                           landmarks[getattr(mp_pose.PoseLandmark, jointsArray[3]).value].y]
            lm_Toe      = [landmarks[getattr(mp_pose.PoseLandmark, jointsArray[4]).value].x,
                           landmarks[getattr(mp_pose.PoseLandmark, jointsArray[4]).value].y]

            hipAngle   = calcAngle(lm_Shoulder, lm_Hip, lm_Knee)
            kneeAngle  = calcAngle(lm_Hip, lm_Knee, lm_Ankle)
            ankleAngle = calcAngle(lm_Knee, lm_Ankle, lm_Toe)


            # boolean joint constraints
            hipBool    = True if hipAngle > 110 else False
            kneeBool   = True if kneeAngle > 110 else False
            ankleBool = True if ankleAngle > 110 else False
            
            # Assuming landmarks and joint angles (hipBool, kneeBool, ankleBool) have been calculated earlier

            errors = 1  # Counter to track the number of errors
            gap = 40
            
            if kneeAngle < 90 and not knee_down:
                knee_down = True            # Set flag indicating the knee is below 90 degrees
            if knee_down and kneeAngle > 110:
                count += 1                  # Increment count when knee goes above 110 after being below 90
                knee_down = False
            
            if landmarks and hipBool and kneeBool and ankleBool and detection:
                # If all joints are in the correct position, display "PERFECT"
                cv2.putText(image, 
                            f"PERFECT  count: {count}", 
                            (50, 60 + errors*gap),  # Adjusted vertical position
                            cv2.FONT_HERSHEY_SIMPLEX, 
                            1, 
                            (0, 255, 0), 
                            2, 
                            cv2.LINE_AA)
                
            elif detection:
                # If there are errors, display them sequentially without overlap
                if not hipBool:
                    errors += 1
                    cv2.putText(image, 
                                f"HIP   : WRONG", 
                                (10, 60 + errors*gap),  # Adjusted vertical position
                                cv2.FONT_HERSHEY_SIMPLEX, 
                                1, 
                                (0, 0, 255), 
                                2, 
                                cv2.LINE_AA)

                if not kneeBool:
                    errors += 1
                    cv2.putText(image, 
                                f"KNEE  : WRONG", 
                                (10, 60 + errors*gap),  # Adjusted vertical position
                                cv2.FONT_HERSHEY_SIMPLEX, 
                                1, 
                                (0, 0, 255), 
                                2, 
                                cv2.LINE_AA)

                if not ankleBool:
                    errors += 1
                    cv2.putText(image, 
                                f"ANKLE : WRONG", 
                                (10, 60 + errors*gap),  # Adjusted vertical position
                                cv2.FONT_HERSHEY_SIMPLEX, 
                                1, 
                                (0, 0, 255), 
                                2, 
                                cv2.LINE_AA)

             
        except:
            pass

        
        #render the processed image
        mp_drawing.draw_landmarks(image, 
                                  results.pose_landmarks, 
                                  mp_pose.POSE_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(
                                      color=(245,117,66), 
                                      thickness=2, 
                                      circle_radius=1),
                                 mp_drawing.DrawingSpec(
                                      color=(245,66,230), 
                                      thickness=2, 
                                      circle_radius=2),
                                 )

        cv2.imshow("Mediapipe Feed", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
        
cap.release()
cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
